In [134]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [135]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


## Parameters

In [136]:
# General constants
g = 9.81 # m/s^2
rho_w = 1025 # kg/m^3
rho_s = 2650 # kg/m^3
nu = 1.33e-6 # m^2/s
bodemdiepte = 40 # mMSL
Kv = 1.4 # Velocity coefficient
Delta_v = (rho_s - rho_w) / rho_w # Dimensionless

## Armour Layer
### Boek methode

Diameter van de armour layer wordt hieronder bepaald via de methode boek: de adpated shield approach. Alleen snelheid (u_c) en waterdiepte (h) zullen varieren tussen weeromstandigheden.
- De waarde van Kv moet nog geverifieerd worden. Of bij docenten of uit literatuur 

In [137]:
def calculate_d_n50(u_c, H, T, h, psi_c, k_r, K_s=1.0, K_v=Kv, delta=Delta_v):
    """
    Calculate the characteristic stone diameter d_n50 for an armour layer (with adapted Shields formula).

    Parameters 
    ----------
    u_c : float
        Current velocity depth-averaged [m/s]
    H : float
        Wave height [m]
    T : float
        Wave period [s]
    h : float
        Water depth [m]
    psi_c : float, optional
        Critical motion parameter [-]
    k_r : float
        bottom_roughness [-]
    K_s : float, optional
        Strength reduction factor for stones on a slope [-]
    K_v : float, optional
        Velocity/turbulence factor [-]
    delta : float, optional
        Relative density (Δ = (rho_s - rho_w) / rho_w) [-]

    Returns
    -------
    d_n50 : float
        Required median stone diameter [m]
    """

    C = 50.0  # initial Chezy coefficient guess
    omega = 2.0 * np.pi / T
    k = omega**2 / g  # Initial guess for wave number
    for _ in range(10):
        k = omega**2 / (g * np.tanh(k * h))  # Update k using dispersion relation

    a_b = (H / 2.0) / np.sinh(k * h)
    u_b_hat = omega * a_b
    c_f = 0.237 * (a_b / k_r)**(-0.52)
    
    u_star_r_mean = np.sqrt((g / C**2) * u_c**2 + (c_f / 2) * u_b_hat**2 * 0.5)
    d_n50 = (K_v**2 * u_star_r_mean**2) / (K_s * psi_c * delta * C**2)

    return u_star_r_mean, d_n50

In [138]:
res_HWL = calculate_d_n50(HWL.CS_total, HWL.Hm0, HWL.Tp, HWL.WL_total + bodemdiepte, 0.3, 0.1)
res_LWL = calculate_d_n50(LWL.CS_total, LWL.Hm0, LWL.Tp, LWL.WL_total + bodemdiepte, 0.3, 0.1)
print(f'For HWL conditions:  \n\t- u_c = {HWL.CS_total} m/s,\n\t- H = {HWL.Hm0} m,\n\t- T = {HWL.Tp} s,\n\t- h = {HWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_HWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_HWL[1]:.5f} m.\n')
print(f'For LWL conditions:  \n\t- u_c = {LWL.CS_total} m/s,\n\t- H = {LWL.Hm0} m,\n\t- T = {LWL.Tp} s,\n\t- h = {LWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_LWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_LWL[1]:.5f} m.')

For HWL conditions:  
	- u_c = 0.5 m/s,
	- H = 9.9 m,
	- T = 15.3 s,
	- h = 42.4 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.1667 m/s
 and the required median stone diameter d_n50is:	 d_n50 = 0.00005 m.

For LWL conditions:  
	- u_c = 0.5 m/s,
	- H = 8.7 m,
	- T = 13.4 s,
	- h = 38.3 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.1561 m/s
 and the required median stone diameter d_n50is:	 d_n50 = 0.00004 m.


### ARMOUR LAYER  HASPRO method



In [160]:
def calc_hydrodynamic_params(T_p, H_s, u_c, h_w, D_n50, D_p, g=9.81):
    T_z = T_p / 1.3 

    u_m_bed = np.sqrt(2) * (H_s / 4) * np.sqrt(g / h_w) * np.exp(-((3.65 / T_z) * np.sqrt(h_w / g))**2.1)

    L_w_peak_period = 150
    for repetition in range(10):
        L_w_peak_period = g * T_p**2 / (2 * np.pi) * np.tanh((2 * np.pi * h_w) / L_w_peak_period)
    
    K_top =  np.sinh(2 * np.pi * h_w / L_w_peak_period) / np.sinh(2 * np.pi * (h_w - 4 * D_n50) / L_w_peak_period)

    u_m_top = u_m_bed * K_top

    A_w_a = (u_m_top * T_p) / (2 * np.pi)

    KC_c = np.abs(u_c) * T_p / D_p 
    KC_w = u_m_bed * T_p / D_p 
    KC_total = KC_c + KC_w

    return u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total , T_z, L_w_peak_period



In [161]:
def bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w):
    k_s = n * d_50

    ampl_ratio = A_w_a / k_s

    wave_friction_coefficient = 0.237 * (ampl_ratio)**(-0.52)

    u_ww = np.sqrt(wave_friction_coefficient / 2) * u_m_top

    tau_bed_wave = rho_w * u_ww**2

    bed_roughness_length = k_s / 30

    drag_coefficient = (0.4 / (np.log(h_w / bed_roughness_length) - 1))**2

    shear_velocity_current = np.sqrt(drag_coefficient) * u_c

    tau_bed_current = rho_w * shear_velocity_current**2

    tau_mean = tau_bed_current + 1.2 * tau_bed_current * (tau_bed_wave / (tau_bed_current + tau_bed_wave))**3.2

    tau_max = tau_mean + tau_bed_wave 

    return wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max 




In [162]:
def mobility_number(rho_stone=rho_stone, rho_w=rho_w, d_50=d_50, g=g, nu=nu, tau_max=tau_max):
    delta_s = (rho_stone - rho_w) / rho_w
    d_star = d_50 * (delta_s * g / nu**2)**(1/3)
    shields_crit = 0.30 / (1 + 1.2 * d_star) + 0.055 * (1 - np.exp(-0.02 * d_star))
    shields_combined = tau_max / (delta_s * g * d_50 * rho_w)
    MOB_top = shields_combined / shields_crit

    return d_star, shields_crit, shields_combined, MOB_top


In [163]:
def depth_deformation(KC_total=KC_total, D_p=D_p, MOB_top=MOB_top):
    
    f_ = 1 + (3.9274 / (1 + np.exp(-0.7401 * KC_total + 4.7518)))
    s_50 = D_p * f_ * 0.0707 * MOB_top**1.6492
    s_90 = D_p * f_ * 0.1134 * MOB_top**1.6492

    return f_, s_50, s_90




In [164]:
def acceptable_s(KC_total = KC_total, acceptable_movement=acceptable_movement):
    if KC_total <= 4.2:
        if acceptable_movement == 'Very limited':
            s_acc_50 = 0.48
            s_acc_90 = 0.34
        elif acceptable_movement == 'Limited':
            s_acc_50 = 0.715
            s_acc_90 = 0.5
        elif acceptable_movement == 'Significant':
            s_acc_50 = 0.895
            s_acc_90 = 0.63
        elif acceptable_movement == 'Extreme':
            s_acc_50 = 1.055
            s_acc_90 = 0.735
    else:
        if acceptable_movement == 'Very limited':
            s_acc_50 = 0.36
            s_acc_90 = 0.235
        elif acceptable_movement == 'Limited':
            s_acc_50 = 0.535
            s_acc_90 = 0.355
        elif acceptable_movement == 'Significant':
            s_acc_50 = 0.675
            s_acc_90 = 0.45
        elif acceptable_movement == 'Extreme':
            s_acc_50 = 0.8
            s_acc_90 = 0.53
    return s_acc_50, s_acc_90


    

In [166]:
acceptable_movement = 'Very limited' # options: 'Very limited', 'Limited', 'Significant', 'Extreme'

def amour_layer_computation(acceptable_movement=acceptable_movement):

    'Computation makes use of parameters chosen by user'

    d_50 = 0.01 # m 
    D_p = 9 # m
    g = 9.81 # m/s^2
    rho_w = 1025 # kg/m^3
    rho_stone = 2650 # kg/m^3 [from stone grading table]
    nu = 1.33e-6 # m^2/s
    H_s = 9.9 # m
    T_p = 10.5 # s
    u_c = 0.5 # m/s
    h_w = 40 # m [water depth at site]
    D_n50 = 0.084 # m 
    n = 2.5 # given constant

    

    for repetition in range(10):

        u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total, T_z, L_w_peak_period = calc_hydrodynamic_params(T_p, H_s, u_c, h_w, D_n50, D_p)

        wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max = bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w)

        d_star, shields_crit, shields_combined, MOB_top = mobility_number(rho_stone=rho_stone, rho_w=rho_w, d_50=d_50, g=g, nu=nu, tau_max=tau_max)

        f_, s_50, s_90 = depth_deformation(KC_total=KC_total, D_p=D_p, MOB_top=MOB_top)

        s_acc_50, s_acc_90 = acceptable_s(KC_total = KC_total, acceptable_movement=acceptable_movement)

        print(f's_50 = {s_50:.3f} m, s_acc_50 = {s_acc_50} m, | s_90 = {s_90:.3f} m, s_acc_90 = {s_acc_90} m')
    
        if s_50 <= s_acc_50 or s_90 <= s_acc_90:
            print(f'\nDesign is acceptabed with d_50 = {d_50:.3f} m')
            break
        else:
            d_50 += 0.01
            D_n50 = d_50 * 0.84 # update D_n50 based on new d_50
            print(f'Increasing d_50 to {d_50:.3f} m and dn50 to {D_n50:.3f} m for next iteration.')
    
    return d_50, s_50, s_90, s_acc_50, s_acc_90

d_50_final, s_50_final, s_90_final, s_acc_50_final, s_acc_90_final = amour_layer_computation()
print(f'\nThe maximum deformation allowed for the chosen acceptable movement "{acceptable_movement}" are:\ns_acc_50 = {s_acc_50_final} m\ns_acc_90 = {s_acc_90_final} m')
print(f'\nFinal design:\nd_50 = {d_50_final:.3f} m\ns_50 = {s_50_final:.3f} m\ns_90 = {s_90_final:.3f} m\ns_acc_50 = {s_acc_50_final} m\ns_acc_90 = {s_acc_90_final} m')
print('\n--- END OF CALCULATION ---')



s_50 = 0.883 m, s_acc_50 = 0.48 m, | s_90 = 1.416 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.020 m and dn50 to 0.017 m for next iteration.
s_50 = 0.480 m, s_acc_50 = 0.48 m, | s_90 = 0.770 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.030 m and dn50 to 0.025 m for next iteration.
s_50 = 0.347 m, s_acc_50 = 0.48 m, | s_90 = 0.557 m, s_acc_90 = 0.34 m

Design is acceptabed with d_50 = 0.030 m

The maximum deformation allowed for the chosen acceptable movement "Very limited" are:
s_acc_50 = 0.48 m
s_acc_90 = 0.34 m

Final design:
d_50 = 0.030 m
s_50 = 0.347 m
s_90 = 0.557 m
s_acc_50 = 0.48 m
s_acc_90 = 0.34 m

--- END OF CALCULATION ---
